In [281]:
import os
import json
import pandas as pd
import traceback

In [158]:
from langchain.chat_models import ChatOpenAI

In [159]:
from dotenv import load_dotenv

load_dotenv()

True

In [160]:
KEY=os.getenv("OPENAI_API_KEY")

In [162]:
llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo",temperature=0.5)

In [164]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2



In [165]:
RESPONSE_JSON={
"1":  {
     "mcq" : "multiple choice questions",
     "options":{
         "a": "choice here",
         "b": "choice here",
         "c": "choice here",
         "d":  "choice here",
     },
"correct": "correct answer",
},

"2" :{
     "mcq": "multiple choice question",
     "options": {
         "a": "choice here",
         "b": "choice here",
         "c": "choice here",
         "d":  "choice here",
         },

    "correct": "correct answer",

     },


"3" :{
     "mcq": "multiple choice question",
     "options": {
         "a": "choice here",
         "b": "choice here",
         "c": "choice here",
         "d":  "choice here",
         },

    "correct": "correct answer",

     },


"4" :{
     "mcq": "multiple choice question",
     "options": {
         "a": "choice here",
         "b": "choice here",
         "c": "choice here",
         "d":  "choice here",
         },

    "correct": "correct answer",

     },


"4" :{
     "mcq": "multiple choice question",
     "options": {
         "a": "choice here",
         "b": "choice here",
         "c": "choice here",
         "d":  "choice here",
         },

    "correct": "correct answer",

     },

}

In [166]:
TEMPLATE = """
Text:{text}
you are an expert MCQ maker. Given the above text, it is your job to \
 
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}




"""

In [167]:
quiz_generation_prompt = PromptTemplate(
input_variables = ["text","number","subject","tone","response_json"],
template=TEMPLATE
)

In [168]:
quiz_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [169]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question amd give a complete analysis of the quiz. Only use at max 50 words for complexity \
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz} 


Check from an expert English writer of the above quiz:
"""

In [170]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject","quiz"], template=TEMPLATE)

In [171]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [172]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text","number","subject","tone","response_json"], output_variables=["quiz", "review"], verbose=True,)

In [205]:
file_path=r"C:\Users\msing\mcqgen1\data.txt"

In [206]:
file_path

'C:\\Users\\msing\\mcqgen1\\data.txt'

In [204]:
with open(file_path, 'r') as file:
 TEXT = file.read()

In [207]:
print(TEXT)

As a scientific endeavor, machine learning grew out of the quest for artificial intelligence (AI). 
In the early days of AI as an academic discipline, some researchers were interested in having machines learn from data. 
They attempted to approach the problem with various symbolic methods, as well as what were then termed "neural networks"; 
these were mostly perceptrons and other models that were later found to be reinventions of the generalized linear models of statistics.
Probabilistic reasoning was also employed, especially in automated medical diagnosis. However, an increasing emphasis on the logical, 
knowledge-based approach caused a rift between AI and machine learning. Probabilistic systems were plagued by theoretical and practical problems of data acquisition and 
representation. By 1980, expert systems had come to dominate AI, and statistics was out of favor.Work on symbolic/knowledge-based learning did continue within AI, 
leading to inductive logic programming(ILP), but th

In [208]:
# Serialize the python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "4": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [213]:
NUMBER=5
SUBJECT="machine learning"
TONE="simple"

In [214]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

# How to setup Token Usage tracking in Langchain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
    {
      "text": TEXT,
       "number": NUMBER,
       "subject": SUBJECT,
       "tone": TONE,
       "response_json": json.dumps(RESPONSE_JSON)

    }
    )




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:As a scientific endeavor, machine learning grew out of the quest for artificial intelligence (AI). 
In the early days of AI as an academic discipline, some researchers were interested in having machines learn from data. 
They attempted to approach the problem with various symbolic methods, as well as what were then termed "neural networks"; 
these were mostly perceptrons and other models that were later found to be reinventions of the generalized linear models of statistics.
Probabilistic reasoning was also employed, especially in automated medical diagnosis. However, an increasing emphasis on the logical, 
knowledge-based approach caused a rift between AI and machine learning. Probabilistic systems were plagued by theoretical and practical problems of data acquisition and 
representation. By 1980, expert systems had come to dominate AI, and statistics was out of favor.Work on sy

In [210]:
print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens: 1821
Prompt Tokens:1096
Completion Tokens:725
Total Cost:0.0030940000000000004


In [217]:
response

{'text': 'As a scientific endeavor, machine learning grew out of the quest for artificial intelligence (AI). \nIn the early days of AI as an academic discipline, some researchers were interested in having machines learn from data. \nThey attempted to approach the problem with various symbolic methods, as well as what were then termed "neural networks"; \nthese were mostly perceptrons and other models that were later found to be reinventions of the generalized linear models of statistics.\nProbabilistic reasoning was also employed, especially in automated medical diagnosis.\u200aHowever, an increasing emphasis on the logical, \nknowledge-based approach caused a rift between AI and machine learning. Probabilistic systems were plagued by theoretical and practical problems of data acquisition and \nrepresentation. By 1980, expert systems had come to dominate AI, and statistics was out of favor.Work on symbolic/knowledge-based learning did continue within AI, \nleading to inductive logic pr

In [263]:
quiz=response.get("quiz")

In [264]:
print(quiz)

### RESPONSE_JSON
{"1": {"mcq": "What was the early focus of some researchers in AI as an academic discipline?", "options": {"a": "Having machines learn from data", "b": "Creating expert systems", "c": "Developing neural networks", "d": "Using symbolic methods"}, "correct": "a"}, 
"2": {"mcq": "What caused a rift between AI and machine learning?", "options": {"a": "Probabilistic reasoning", "b": "Logical, knowledge-based approach", "c": "Symbolic methods", "d": "Generalized linear models"}, "correct": "b"}, 
"3": {"mcq": "Which research line was continued outside the AI/CS field as 'connectionism'?", "options": {"a": "Expert systems", "b": "Inductive logic programming", "c": "Neural networks", "d": "Symbolic methods"}, "correct": "c"}, 
"4": {"mcq": "What dominated AI by 1980, pushing statistics out of favor?", "options": {"a": "Inductive logic programming", "b": "Neural networks", "c": "Expert systems", "d": "Pattern recognition"}, "correct": "c"}, 
"5": {"mcq": "Which method was plag

In [265]:
print(repr(quiz))

'### RESPONSE_JSON\n{"1": {"mcq": "What was the early focus of some researchers in AI as an academic discipline?", "options": {"a": "Having machines learn from data", "b": "Creating expert systems", "c": "Developing neural networks", "d": "Using symbolic methods"}, "correct": "a"}, \n"2": {"mcq": "What caused a rift between AI and machine learning?", "options": {"a": "Probabilistic reasoning", "b": "Logical, knowledge-based approach", "c": "Symbolic methods", "d": "Generalized linear models"}, "correct": "b"}, \n"3": {"mcq": "Which research line was continued outside the AI/CS field as \'connectionism\'?", "options": {"a": "Expert systems", "b": "Inductive logic programming", "c": "Neural networks", "d": "Symbolic methods"}, "correct": "c"}, \n"4": {"mcq": "What dominated AI by 1980, pushing statistics out of favor?", "options": {"a": "Inductive logic programming", "b": "Neural networks", "c": "Expert systems", "d": "Pattern recognition"}, "correct": "c"}, \n"5": {"mcq": "Which method 

In [266]:
quiz=quiz.strip()  # Removes leading and trailing whitespace or special characters

In [269]:
#quiz_data = json.loads(quiz)

In [ ]:
#json.loads(quiz)

In [270]:
import json

# Check the original quiz string
print("Original quiz string:", repr(quiz))

# Clean the string by removing '### RESPONSE_JSON\n'
cleaned_quiz = quiz.replace('### RESPONSE_JSON\n', '').strip()

# Print the cleaned string for verification
print("Cleaned quiz string:", repr(cleaned_quiz))

# Load the JSON
try:
    quiz_data = json.loads(cleaned_quiz)
    print("Parsed JSON data:", quiz_data)
except json.JSONDecodeError as e:
    print(f"JSONDecodeError: {e}")

Original quiz string: '### RESPONSE_JSON\n{"1": {"mcq": "What was the early focus of some researchers in AI as an academic discipline?", "options": {"a": "Having machines learn from data", "b": "Creating expert systems", "c": "Developing neural networks", "d": "Using symbolic methods"}, "correct": "a"}, \n"2": {"mcq": "What caused a rift between AI and machine learning?", "options": {"a": "Probabilistic reasoning", "b": "Logical, knowledge-based approach", "c": "Symbolic methods", "d": "Generalized linear models"}, "correct": "b"}, \n"3": {"mcq": "Which research line was continued outside the AI/CS field as \'connectionism\'?", "options": {"a": "Expert systems", "b": "Inductive logic programming", "c": "Neural networks", "d": "Symbolic methods"}, "correct": "c"}, \n"4": {"mcq": "What dominated AI by 1980, pushing statistics out of favor?", "options": {"a": "Inductive logic programming", "b": "Neural networks", "c": "Expert systems", "d": "Pattern recognition"}, "correct": "c"}, \n"5": 

In [271]:
json.loads(cleaned_quiz)

{'1': {'mcq': 'What was the early focus of some researchers in AI as an academic discipline?',
  'options': {'a': 'Having machines learn from data',
   'b': 'Creating expert systems',
   'c': 'Developing neural networks',
   'd': 'Using symbolic methods'},
  'correct': 'a'},
 '2': {'mcq': 'What caused a rift between AI and machine learning?',
  'options': {'a': 'Probabilistic reasoning',
   'b': 'Logical, knowledge-based approach',
   'c': 'Symbolic methods',
   'd': 'Generalized linear models'},
  'correct': 'b'},
 '3': {'mcq': "Which research line was continued outside the AI/CS field as 'connectionism'?",
  'options': {'a': 'Expert systems',
   'b': 'Inductive logic programming',
   'c': 'Neural networks',
   'd': 'Symbolic methods'},
  'correct': 'c'},
 '4': {'mcq': 'What dominated AI by 1980, pushing statistics out of favor?',
  'options': {'a': 'Inductive logic programming',
   'b': 'Neural networks',
   'c': 'Expert systems',
   'd': 'Pattern recognition'},
  'correct': 'c'},
 '

In [229]:
print(type(cleaned_quiz))

<class 'str'>


In [272]:

import json
# Parse the cleaned JSON string
cleaned_quiz_str = quiz.replace("### RESPONSE_JSON\n", "")  # Remove the prefix

cleaned_quiz = json.loads(cleaned_quiz_str)

# Check the type of cleaned_quiz
print("Type of cleaned_quiz:", type(cleaned_quiz))

# Print the cleaned_quiz to see its content
print("Contents of cleaned_quiz:", cleaned_quiz)

Type of cleaned_quiz: <class 'dict'>
Contents of cleaned_quiz: {'1': {'mcq': 'What was the early focus of some researchers in AI as an academic discipline?', 'options': {'a': 'Having machines learn from data', 'b': 'Creating expert systems', 'c': 'Developing neural networks', 'd': 'Using symbolic methods'}, 'correct': 'a'}, '2': {'mcq': 'What caused a rift between AI and machine learning?', 'options': {'a': 'Probabilistic reasoning', 'b': 'Logical, knowledge-based approach', 'c': 'Symbolic methods', 'd': 'Generalized linear models'}, 'correct': 'b'}, '3': {'mcq': "Which research line was continued outside the AI/CS field as 'connectionism'?", 'options': {'a': 'Expert systems', 'b': 'Inductive logic programming', 'c': 'Neural networks', 'd': 'Symbolic methods'}, 'correct': 'c'}, '4': {'mcq': 'What dominated AI by 1980, pushing statistics out of favor?', 'options': {'a': 'Inductive logic programming', 'b': 'Neural networks', 'c': 'Expert systems', 'd': 'Pattern recognition'}, 'correct': 

In [273]:
# Lets try to create a dataframe by using the above dictionary:

quiz_table_data =[]
for key,value in cleaned_quiz.items():
    mcq= value["mcq"]
options = "|".join(
[
   f"{option}:{option_value}"
    for option, option_value in value["options"].items()
    ]
)
correct = value["correct"]
quiz_table_data.append({"MCQ":mcq, "choices":options, "Correct":correct})


In [274]:
quiz_table_data

[{'MCQ': 'Which method was plagued by theoretical and practical problems of data acquisition and representation?',
  'choices': 'a:Probabilistic reasoning|b:Symbolic methods|c:Neural networks|d:Generalized linear models',
  'Correct': 'a'}]

In [276]:
import json

# Assuming you have the cleaned quiz string
cleaned_quiz_str = quiz.replace("### RESPONSE_JSON\n", "")  # Clean the input string
cleaned_quiz = json.loads(cleaned_quiz_str)  # Parse the JSON string

# Debug: Check the contents of cleaned_quiz
print("Cleaned Quiz Data:", cleaned_quiz)

# Check if cleaned_quiz is a dictionary
if isinstance(cleaned_quiz, dict):
    # Prepare the quiz table data
    quiz_table_data = []
    
    for key, value in cleaned_quiz.items():
        # Ensure that value has the expected keys
        if "mcq" in value and "options" in value and "correct" in value:
            mcq = value["mcq"]
            options = "|".join(
                [f"{option}:{option_value}" for option, option_value in value["options"].items()]
            )
            correct = value["correct"]

            # Debug: Print each MCQ and its corresponding values
            print(f"MCQ {key}: {mcq}, Choices: {options}, Correct: {correct}")

            # Append to the quiz table data
            quiz_table_data.append({"MCQ": mcq, "choices": options, "Correct": correct})
        else:
            print(f"Warning: Expected keys not found in entry {key}")

    # Print the resulting quiz_table_data
    print("Quiz Table Data:", quiz_table_data)
    print(f"Total number of MCQs: {len(quiz_table_data)}")
else:
    print("Error: cleaned_quiz is not a dictionary.")

Cleaned Quiz Data: {'1': {'mcq': 'What was the early focus of some researchers in AI as an academic discipline?', 'options': {'a': 'Having machines learn from data', 'b': 'Creating expert systems', 'c': 'Developing neural networks', 'd': 'Using symbolic methods'}, 'correct': 'a'}, '2': {'mcq': 'What caused a rift between AI and machine learning?', 'options': {'a': 'Probabilistic reasoning', 'b': 'Logical, knowledge-based approach', 'c': 'Symbolic methods', 'd': 'Generalized linear models'}, 'correct': 'b'}, '3': {'mcq': "Which research line was continued outside the AI/CS field as 'connectionism'?", 'options': {'a': 'Expert systems', 'b': 'Inductive logic programming', 'c': 'Neural networks', 'd': 'Symbolic methods'}, 'correct': 'c'}, '4': {'mcq': 'What dominated AI by 1980, pushing statistics out of favor?', 'options': {'a': 'Inductive logic programming', 'b': 'Neural networks', 'c': 'Expert systems', 'd': 'Pattern recognition'}, 'correct': 'c'}, '5': {'mcq': 'Which method was plagued

In [277]:
quiz_table_data

[{'MCQ': 'What was the early focus of some researchers in AI as an academic discipline?',
  'choices': 'a:Having machines learn from data|b:Creating expert systems|c:Developing neural networks|d:Using symbolic methods',
  'Correct': 'a'},
 {'MCQ': 'What caused a rift between AI and machine learning?',
  'choices': 'a:Probabilistic reasoning|b:Logical, knowledge-based approach|c:Symbolic methods|d:Generalized linear models',
  'Correct': 'b'},
 {'MCQ': "Which research line was continued outside the AI/CS field as 'connectionism'?",
  'choices': 'a:Expert systems|b:Inductive logic programming|c:Neural networks|d:Symbolic methods',
  'Correct': 'c'},
 {'MCQ': 'What dominated AI by 1980, pushing statistics out of favor?',
  'choices': 'a:Inductive logic programming|b:Neural networks|c:Expert systems|d:Pattern recognition',
  'Correct': 'c'},
 {'MCQ': 'Which method was plagued by theoretical and practical problems of data acquisition and representation?',
  'choices': 'a:Probabilistic reaso

In [284]:
quiz= pd.DataFrame(quiz_table_data)

In [285]:
quiz

,MCQ,choices,Correct
0,What was the early focus of some researchers i...,a:Having machines learn from data|b:Creating e...,a
1,What caused a rift between AI and machine lear...,"a:Probabilistic reasoning|b:Logical, knowledge...",b
2,Which research line was continued outside the ...,a:Expert systems|b:Inductive logic programming...,c
3,"What dominated AI by 1980, pushing statistics ...",a:Inductive logic programming|b:Neural network...,c
4,Which method was plagued by theoretical and pr...,a:Probabilistic reasoning|b:Symbolic methods|c...,a


In [286]:
quiz.to_csv("machinelearning.csv", index=False)

In [1]:
python git-filter-repo.py --path experiment/mcq.ipynb --invert-paths


SyntaxError: invalid syntax (1421987323.py, line 1)